## Import necessary modules
Run this cell before running any other cells

In [6]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Printing and Logging
## Printing
You can use the **print()** function in Python to print messages to the screen. <br>
The message can be a string, or any other object, the object will be converted into a string before it is written to the screen. <br>

## Logging
You could use the logging module that is setup in *utils.py*. <br>
It prints to both your screen (standard output) as well as to log files (*ble.log*) in the *logs* directory. <br>
This is the recommended way to output messages, since the log files can help with debugging. <br>
The logging module also provides different log levels as shown below, each formatted with a different color for increased visibility. <br>

__**NOTE**__: You may notice that the DEBUG message is not printed to the screen but is printed in the log file. This is because the logging level for the screen is set to INFO and for the file is set to DEBUG. You can change the default log levels in *utils.py* (**STREAM_LOG_LEVEL** and **FILE_LOG_LEVEL**). 

## Formatting output
To format your strings, you may use %-formatting, str.format() or f-strings. <br>
The most "pythonic" way would be to use f-strings. [Here](https://realpython.com/python-f-strings/) is a good tutorial on f-strings. <br>

In [30]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

2023-02-11 13:11:48,985 | INFO     |: info
2023-02-11 13:11:48,986 | WARNING  |: warning
2023-02-11 13:11:48,986 | ERROR    |: error
2023-02-11 13:11:48,987 | CRITICAL |: critical


<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [96]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2023-02-21 22:28:40,691 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:9:6a:32:3c
2023-02-21 22:28:43,438 | INFO     |: Connected to c0:83:9:6a:32:3c


## Receive data from the Artemis board

The cell below shows examples of reading different types (as defined in the Arduino sketch) of GATT characteristics.

In [3]:
# Read a float GATT Charactersistic
f = ble.receive_float(ble.uuid['RX_FLOAT'])
print(f)

125.0


In [4]:
# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

[->9.0<-]


## Send a command to the Artemis board
Send the PING command and read the reply string from the string characteristic RX_STRING. <br>
__NOTE__: The **send_command()** essentially sends a string data to the GATT characteristic (TX_CMD_STRING). The GATT characteristic in the Arduino sketch is of type BLECStringCharacteristic.

In [5]:
ble.send_command(CMD.PING, "")

In [6]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

PONG


The cell below shows an example of the SEND_TWO_INTS command. <br> The two values in the **data** are separated by a delimiter "|". <br>
Refer Lab 2 documentation for more information on the command protocol.

In [7]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

The Artemis board should print the two integers to the serial monitor in the ArduinoIDE. 

In [9]:
ble.send_command(CMD.ECHO, "HiHello")

In [6]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

HiHello:)


In [8]:
ble.send_command(CMD.GET_TIME_MILLIS, "")

In [9]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

T:264263


In [24]:
string = ''
def callback(uuid, byte_array):
    global string
    string = ble.bytearray_to_string(byte_array)[2:]
    
ble.start_notify(ble.uuid['RX_STRING'], callback)

In [26]:
ble.send_command(CMD.GET_TIME_MILLIS, "")
print(string)

347988


In [27]:
ble.stop_notify(ble.uuid['RX_STRING'])

### GET_TEMP_5S

In [4]:
string = ''
def callback(uuid, byte_array):
    global string
    string = ble.bytearray_to_string(byte_array)
    
ble.start_notify(ble.uuid['RX_STRING'], callback)

In [43]:
ble.send_command(CMD.GET_TEMP_5s, "")
print(string)

T:637683|C:23.790|T:638683|C:23.790|T:639683|C:23.790|T:640683|C:23.790|T:641683|C:23.198|


In [101]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [45]:
temps = ble.receive_string(ble.uuid['RX_STRING'])
print(temps)

T:642683|C:23.790|T:643683|C:23.198|T:644683|C:23.198|T:645683|C:23.790|T:646683|C:23.790|


### GET_TEMP_5S_RAPID

In [102]:
rapid_temps = []
def callback(uuid, byte_array):
    global string
    global rapid_temps
    string = ble.bytearray_to_string(byte_array)
    rapid_temps.append(string)
    
ble.start_notify(ble.uuid['RX_STRING'], callback)

In [103]:
ble.send_command(CMD.GET_TEMP_5s_RAPID, "")
print(rapid_temps)

[]


In [31]:
num_samples = len(rapid_temps)
print(num_samples)

969


In [104]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [3]:
rapid_temps = ble.receive_string(ble.uuid['RX_STRING'])
print(rapid_temps)

[->9.0<-]


In [105]:
data_list = []
def callback(uuid, byte_array):
    global string
    global data_list
    string = ble.bytearray_to_string(byte_array)
    data_list.append(string)
    
ble.start_notify(ble.uuid['RX_STRING'], callback)

In [106]:
ble.send_command(CMD.GET_TOF_IMU, "")

In [107]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(data_list)

['T:204412|Sensor 1:2072|Sensor 2:0|', 'T:204412|Sensor 1:2072|Sensor 2:0|Pitch:-11864.798|Roll:-5.44|', 'T:204433|Sensor 1:2072|Sensor 2:0|', 'T:204433|Sensor 1:2072|Sensor 2:0|Pitch:-5931.389|Roll:-2.940|', 'T:204454|Sensor 1:2320|Sensor 2:0|', 'T:204454|Sensor 1:2320|Sensor 2:0|Pitch:-2964.444|Roll:-1.752|', 'T:204471|Sensor 1:2320|Sensor 2:0|', 'T:204471|Sensor 1:2320|Sensor 2:0|Pitch:-1480.609|Roll:-1.132|', 'T:204489|Sensor 1:0|Sensor 2:0|', 'T:204489|Sensor 1:0|Sensor 2:0|Pitch:-738.909|Roll:-1.99|', 'T:204510|Sensor 1:0|Sensor 2:0|', 'T:204510|Sensor 1:0|Sensor 2:0|Pitch:-368.351|Roll:0.698|', 'T:204529|Sensor 1:0|Sensor 2:0|', 'T:204529|Sensor 1:0|Sensor 2:0|Pitch:-183.47|Roll:0.714|', 'T:204560|Sensor 1:235|Sensor 2:0|', 'T:204560|Sensor 1:235|Sensor 2:0|Pitch:-91.140|Roll:0.830|', 'T:204577|Sensor 1:235|Sensor 2:0|', 'T:204577|Sensor 1:235|Sensor 2:0|Pitch:-44.608|Roll:0.792|', 'T:204610|Sensor 1:285|Sensor 2:0|', 'T:204610|Sensor 1:285|Sensor 2:0|Pitch:-23.182|Roll:0.610|',

## Disconnect

In [76]:
# Disconnect
ble.disconnect()

2023-02-21 22:24:54,362 | INFO     |: Disconnected from 696B5A40-E16D-9AF3-A22D-7B719373B996
